In [14]:
!pip install pandas chatterbot chatterbot_corpus 

Defaulting to user installation because normal site-packages is not writeable


In [15]:
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/12.8 MB 13.4 MB/s eta 0:00:01
     ------ --------------------------------- 2.1/12.8 MB 7.3 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.8 MB 5.4 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 4.7 MB/s eta 0:00:03
     ------------ --------------------------- 3.9/12.8 MB 4.3 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 4.2 MB/s eta 0:00:02
     --------------- ------------------------ 5.0/12.8 MB 3.8 MB/s eta 0:00:03
     ----------------- ---------------------- 5.5/12.8 MB 3.7 MB/s eta 0:00:02
     ------------------ --------------------- 5.8/12.8 MB 3.4 MB/s eta 0:00:03
     ------------------ --------------------- 6.0/12.8 MB 3.2 MB/s eta 0:00:03
     -------------------- ------------------- 6.6/12.8 MB 3

In [16]:
import pandas as pd

# Load the dataset
df = pd.read_csv('dataset-tickets-multi-lang-4-20k.csv')

# Filter for English entries and select question (body) and response (answer) columns
df_en = df[df['language'] == 'en'][['body', 'answer']].rename(columns={'body': 'question', 'answer': 'response'})

# Display the first few rows to verify
df_en.head()

,question,response
1,Seeking information on digital strategies that...,We offer a variety of digital strategies and s...
2,I am contacting you to request information on ...,I am here to assist you with data analytics to...
4,"Dear Customer Support, I am reaching out to in...","Dear [name], we take the security of medical d..."
5,Inquiring about best practices for securing me...,Thank you for your concern regarding securing ...
7,"The integration stopped working unexpectedly, ...",I will look into the problem and call you at <...


In [ ]:
# Remove placeholders and clean text
df_en['question'] = df_en['question'].str.replace(r'<tel_num>|<acc_num>|<name>', 'support', regex=True)
df_en['response'] = df_en['response'].str.replace(r'<tel_num>|<acc_num>|<name>', 'support', regex=True)

# Optionally, remove rows with missing values
df_en = df_en.dropna()

# Save a subset (e.g., 2000 rows) for the demo to keep training fast
df_en_subset = df_en.head(20)
df_en_subset.to_csv('qa_pairs.csv', index=False)

# Display the cleaned subset
df_en_subset.head()

,question,response
1,Seeking information on digital strategies that...,We offer a variety of digital strategies and s...
2,I am contacting you to request information on ...,I am here to assist you with data analytics to...
4,"Dear Customer Support, I am reaching out to in...","Dear [name], we take the security of medical d..."
5,Inquiring about best practices for securing me...,Thank you for your concern regarding securing ...
7,"The integration stopped working unexpectedly, ...",I will look into the problem and call you at s...


In [18]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

# Load the cleaned dataset
df = pd.read_csv('qa_pairs.csv')

# Initialize the chatbot
chatbot = ChatBot('ClassDemoBot')

# Create a trainer
trainer = ListTrainer(chatbot)

# Train the chatbot with the Q&A pairs
for index, row in df.iterrows():
    trainer.train([row['question'], row['response']])

print("Training complete!")

List Trainer: 2it [00:00, 105.67it/s]
List Trainer: 2it [00:00, 84.88it/s]
List Trainer: 2it [00:00, 74.36it/s]
List Trainer: 2it [00:00, 111.21it/s]
List Trainer: 2it [00:00, 163.42it/s]
List Trainer: 2it [00:00, 88.18it/s]
List Trainer: 2it [00:00, 102.76it/s]
List Trainer: 2it [00:00, 72.38it/s]
List Trainer: 2it [00:00, 123.36it/s]
List Trainer: 2it [00:00, 257.68it/s]
List Trainer: 2it [00:00, 127.49it/s]
List Trainer: 2it [00:00, 137.17it/s]
List Trainer: 2it [00:00, 137.06it/s]
List Trainer: 2it [00:00, 380.54it/s]
List Trainer: 2it [00:00, 62.68it/s]
List Trainer: 2it [00:00, 90.04it/s]
List Trainer: 2it [00:00, 215.56it/s]
List Trainer: 2it [00:00, 153.19it/s]
List Trainer: 2it [00:00, 130.60it/s]
List Trainer: 2it [00:00, 105.90it/s]
List Trainer: 2it [00:00, 70.46it/s]
List Trainer: 2it [00:00, 85.55it/s]
List Trainer: 2it [00:00, 103.53it/s]
List Trainer: 2it [00:00, 73.85it/s]
List Trainer: 2it [00:00, 76.48it/s]
List Trainer: 2it [00:00, 122.06it/s]
List Trainer: 2it [00:

Training complete!


In [19]:
# Start a conversation loop
print("Type 'quit' or 'exit' to stop.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit']:
        print("Goodbye!")
        break
    response = chatbot.get_response(user_input)
    print("Bot: ", response)

Type 'quit' or 'exit' to stop.
Bot:  Hello! How can I assist you today?
Bot:  I will examine the integration of TensorFlow and Red Hat for our investment analytics platform to improve decision-making capacities. I will reach you at support to discuss further implementation specifics and estimated timelines.
Bot:  Dear [Your Name], we thank you for bringing to our attention the difficulties you are facing with the integration of multiple applications. We regret the inconvenience this has caused you and appreciate the efforts you have already made to troubleshoot the issue. To better understand and resolve the problem, could you please provide us with additional details regarding the API adjustments or server overload that you suspect may be the cause? It would also be beneficial for us to know the specific error messages you are observing in the logs. If necessary, we can arrange a call at a date and time convenient for you to discuss this further; please notify us at the phone number p

# Issues

## It Struggles with Random Inputs

The dataset primarily contains professional queries (e.g., "Seeking information on digital strategies...") and responses (e.g., "We offer a variety of digital strategies..."). It doesn’t include casual greetings or small talk like "Hi" or "How are you?".

ChatterBot relies on pattern matching and similarity to the training data. If the input doesn’t resemble any trained question, the chatbot may give an irrelevant or random response.

ChatterBot uses a simple matching algorithm (like cosine similarity) to find the closest question in its training data. For "Hi," there’s likely no close match, so it picks a random or poorly matched response.

Without a fallback mechanism or general conversational training, it struggles with out-of-domain inputs.

A few approaches to improve the chatbot’s ability to respond to casual inputs like "Hi" and "How are you?" while keeping your demo setup simple.

In [8]:
import pandas as pd

# Load the existing qa_pairs.csv
df = pd.read_csv('qa_pairs.csv')

# Define casual conversation pairs
casual_pairs = [
    {'question': 'Hi', 'response': 'Hello! How can I assist you today?'},
    {'question': 'Hello', 'response': 'Hi there! What can I help you with?'},
    {'question': 'How are you?', 'response': 'I’m doing great, thanks for asking! How about you?'},
    {'question': 'What’s up?', 'response': 'Not much, just here to help you! What’s on your mind?'},
    {'question': 'Hey', 'response': 'Hey! How can I help you today?'}
]

# Create a DataFrame for casual pairs
casual_df = pd.DataFrame(casual_pairs)

# Append casual pairs to the existing dataset
df_updated = pd.concat([df, casual_df], ignore_index=True)

# Save the updated dataset
df_updated.to_csv('qa_pairs_updated.csv', index=False)

# Verify the updated dataset
print(df_updated.tail())

          question                                           response
2000            Hi                 Hello! How can I assist you today?
2001         Hello                Hi there! What can I help you with?
2002  How are you?  I’m doing great, thanks for asking! How about ...
2003    What’s up?  Not much, just here to help you! What’s on you...
2004           Hey                     Hey! How can I help you today?


In [20]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

# Load the updated dataset
df = pd.read_csv('qa_pairs_updated.csv')

# Initialize the chatbot (clear previous training by recreating it)
chatbot = ChatBot('ClassDemoBot', storage_adapter='chatterbot.storage.SQLStorageAdapter')

# Create a trainer
trainer = ListTrainer(chatbot)

# Train with the updated Q&A pairs
for index, row in df.iterrows():
    trainer.train([row['question'], row['response']])

print("Training complete!")

List Trainer: 2it [00:00, 93.31it/s]
List Trainer: 2it [00:00, 84.61it/s]
List Trainer: 2it [00:00, 74.27it/s]
List Trainer: 2it [00:00, 106.52it/s]
List Trainer: 2it [00:00, 159.43it/s]
List Trainer: 2it [00:00, 85.55it/s]
List Trainer: 2it [00:00, 108.56it/s]
List Trainer: 2it [00:00, 71.13it/s]
List Trainer: 2it [00:00, 127.91it/s]
List Trainer: 2it [00:00, 245.96it/s]
List Trainer: 2it [00:00, 139.82it/s]
List Trainer: 2it [00:00, 159.32it/s]
List Trainer: 2it [00:00, 142.27it/s]
List Trainer: 2it [00:00, 446.06it/s]
List Trainer: 2it [00:00, 64.44it/s]
List Trainer: 2it [00:00, 98.35it/s]
List Trainer: 2it [00:00, 319.70it/s]
List Trainer: 2it [00:00, 190.71it/s]
List Trainer: 2it [00:00, 120.39it/s]
List Trainer: 2it [00:00, 108.57it/s]
List Trainer: 2it [00:00, 87.42it/s]
List Trainer: 2it [00:00, 106.40it/s]
List Trainer: 2it [00:00, 95.03it/s]
List Trainer: 2it [00:00, 74.65it/s]
List Trainer: 2it [00:00, 69.41it/s]
List Trainer: 2it [00:00, 131.21it/s]
List Trainer: 2it [00:0

Training complete!


In [ ]:
# Start a conversation loop
print("Type 'quit' or 'exit' to stop.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['quit', 'exit']:
        print("Goodbye!")
        break
    response = chatbot.get_response(user_input)
    print("Bot: ", response)

In [22]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
import pandas as pd

# Load your dataset
df = pd.read_csv('qa_pairs.csv')

# Initialize the chatbot
chatbot = ChatBot('ClassDemoBot', storage_adapter='chatterbot.storage.SQLStorageAdapter')

# Train with the ChatterBot English corpus for general conversation
corpus_trainer = ChatterBotCorpusTrainer(chatbot)
corpus_trainer.train('chatterbot.corpus.english.greetings', 'chatterbot.corpus.english.conversations')

# Train with your dataset
trainer = ListTrainer(chatbot)
for index, row in df.iterrows():
    trainer.train([row['question'], row['response']])

print("Training complete!")

ChatterBot Corpus Trainer: 2it [00:00,  7.16it/s]
List Trainer: 2it [00:00, 165.80it/s]
List Trainer: 2it [00:00, 94.60it/s]
List Trainer: 2it [00:00, 80.01it/s]
List Trainer: 2it [00:00, 119.63it/s]
List Trainer: 2it [00:00, 153.17it/s]
List Trainer: 2it [00:00, 86.52it/s]
List Trainer: 2it [00:00, 107.73it/s]
List Trainer: 2it [00:00, 72.14it/s]
List Trainer: 2it [00:00, 132.01it/s]
List Trainer: 2it [00:00, 265.08it/s]
List Trainer: 2it [00:00, 125.81it/s]
List Trainer: 2it [00:00, 152.00it/s]
List Trainer: 2it [00:00, 119.88it/s]
List Trainer: 2it [00:00, 397.88it/s]
List Trainer: 2it [00:00, 63.34it/s]
List Trainer: 2it [00:00, 96.67it/s]
List Trainer: 2it [00:00, 233.41it/s]
List Trainer: 2it [00:00, 176.93it/s]
List Trainer: 2it [00:00, 122.56it/s]
List Trainer: 2it [00:00, 107.82it/s]
List Trainer: 2it [00:00, 81.14it/s]
List Trainer: 2it [00:00, 93.91it/s]
List Trainer: 2it [00:00, 82.52it/s]
List Trainer: 2it [00:00, 67.42it/s]
List Trainer: 2it [00:00, 75.31it/s]
List Traine

Training complete!


In [24]:

# Start a conversation loop with a confidence threshold for fallback responses
print("Type 'quit' or 'exit' to stop.")

while True:
    
    # Get user input
    user_input = input("You: ")
    
    # Check for exit condition
    if user_input.lower() in ['quit', 'exit']:
        print("Goodbye!")
        break
    
    # Get the chatbot's response
    response = chatbot.get_response(user_input)
    
    # Check confidence score and provide a fallback if it's too low
    if response.confidence < 0.1:
        print("Bot: I'm not sure how to respond to that. Can you ask something related to customer support, security, or brand growth?")
    else:
        print("Bot: ", response)


Type 'quit' or 'exit' to stop.
Bot:  Hello! How can I assist you today?
Bot:  I’m doing great, thanks for asking! How about you?
Bot:  Right this way.
Bot: I'm not sure how to respond to that. Can you ask something related to customer support, security, or brand growth?
Goodbye!
